In [1]:
import json
import re


In [2]:
with open("all_commands.json", "r") as file:
    all_commands = json.load(file)


In [3]:
all_commands[5]


{'command': 'cyber migrate',
 'Description': ['Migrate the source genesis into the target version and print to STDOUT.'],
 'Example': ['$ cyber migrate v0.36 /path/to/genesis.json --chain-id=cosmoshub-3 --genesis-time=2019-04-22T17:00:00Z'],
 'Usage': ['  cyber migrate [target-version] [genesis-file] [flags]'],
 'Flags': ['      --chain-id string       override chain_id with this flag',
  '      --genesis-time string   override genesis_time with this flag',
  '  -h, --help                  help for migrate'],
 'Global Flags': ['      --home string         directory for config and data (default "/Users/user/.cyber")',
  '      --log_format string   The logging format (json|plain) (default "plain")',
  '      --log_level string    The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")',
  '      --trace               print out full stack trace on errors']}

In [4]:
# n = 0
# for i in all_commands:
#     if i['command'] == 'cyber query evidence':
#         print(n)

#     n += 1

# del all_commands[40]


In [5]:
count = 0
for i in all_commands:
    all_flags = []
    all_flags.extend(i["Flags"])
    if "Global Flags" in i:
        all_flags.extend(i["Global Flags"])

    if all_flags != []:
        i["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [6]:
for i in all_commands:
    print(i["command"], "\n", i["flags_parsed"])


cyber  
 [{'long': 'help', 'short': 'h', 'format': None, 'description': 'help for cyber'}, {'long': 'home', 'short': None, 'format': 'string', 'description': 'directory for config and data (default "/Users/user/.cyber")'}, {'long': 'log_format', 'short': None, 'format': 'string', 'description': 'The logging format (json|plain) (default "plain")'}, {'long': 'log_level', 'short': None, 'format': 'string', 'description': 'The logging level (trace|debug|info|warn|error|fatal|panic) (default "info")'}, {'long': 'trace', 'short': None, 'format': None, 'description': 'print out full stack trace on errors'}]
cyber collect-gentxs 
 [{'long': 'gentx-dir', 'short': None, 'format': 'string', 'description': 'override default "gentx" directory from which collect and execute genesis transactions; default [--home]/config/gentx/'}, {'long': 'help', 'short': 'h', 'format': None, 'description': 'help for collect-gentxs'}, {'long': 'home', 'short': None, 'format': 'string', 'description': 'directory for c

In [7]:
from asyncore import write


types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    'ints': 'string'
}
lines = []

for c in all_commands:
    lines.append(f"\n# {c['Description']}\nextern '{c['command']}' [\n")
    # print(f"\n# {c['Description']}\nextern '{c['command']}' [")
    for f in c["flags_parsed"]:
        if "." not in f["long"]:
            long = f["long"]
            short = "(-" + f["short"] + ")" if f["short"] else ""
            f1 = types_nu_dict.get(f["format"], f["format"])
            format = ": " + f1 if f1 else ""
            description = f["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
            # print(f"\t--{long}{short}{format}\t\t# {description}")
    lines.append("]")
    # print("]")

with open("cyber-autocompletions.nu", "w") as file:
    file.writelines(lines)


In [8]:
import pandas as pd


In [ ]:
df = pd.DataFrame(all_commands).sort_values("command", ascending=False)


In [ ]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()


In [ ]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))


In [ ]:
_list = []

for _file in os.scandir(
    "/Users/user/apps-files/github/cyber_completitions/json-files/"
):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))


In [ ]:
[i for i in _list if i != []]


In [ ]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df


In [ ]:
df["com1"] = "cyber " + df["_command_root"] + " " + df["_command"]
df


In [ ]:
_l2 = df.loc[:, ["_description", "com1"]].to_dict("records")
_l2


In [ ]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')


In [ ]:
for i in json.loads(content):
    print("\t", i["_command"], "\t\t#", i["_description"])
